In [19]:
# Get Elastic Search tools
import numpy as np
import matplotlib.pyplot as plt
import datetime
import re
import json
import sys
from elasticsearch import Elasticsearch, exceptions as es_exceptions
from elasticsearch.helpers import scan
es = Elasticsearch(hosts=[{'host':'atlas-kibana.mwt2.org', 'port':9200}],timeout=60)

In [20]:
# Defining the time period
endtime_start_unix=1478822400
endtime_final_unix=1478995200
# Format for the elastic search
endtime_start = datetime.datetime.fromtimestamp(int(endtime_start_unix)).strftime('%Y-%m-%dT%H:%M:%S.000Z')
endtime_final = datetime.datetime.fromtimestamp(int(endtime_final_unix)).strftime('%Y-%m-%dT%H:%M:%S.000Z')

print (endtime_start)
print (endtime_final)

2016-11-10T18:00:00.000Z
2016-11-12T18:00:00.000Z


In [ ]:
myquery = {
 "size": 50,
 "query": {
   "filtered": {
     "query": {
       "query_string": {
         #"query": "endtime:[endtime_start TO endtime_final] AND (computingsite:CERN-PROD_T0_8MCORE OR computingsite:CERN-PROD_T0_4MCORE)",
         "query": "*",
         "analyze_wildcard": True,
         "lowercase_expanded_terms": False,
       }
     }
   }
 },
 "fields": [
   "modificationtime","pandaid","computingsite","endtime"
 ]
}

In [26]:
myquery = {
    "size": 100,
    "query": {
        "bool": {
            "must": [
                {
                    "range":{"modificationtime": { "gte": 1483284870000, "lte": 1485617722000, "format": "epoch_millis" } }
                }
          #{"term": {"computingsite": "CERN-PROD_T0_8MCORE"}},
          #{"term": {"computingsite": "CERN-PROD_SHORT"}}
            ]
        }
    }
}

In [29]:
res = es.search(index='jobs_archive_2017-*', body=myquery, request_timeout=600)
print(res['hits']['hits'])

[{'_score': 1.0, '_index': 'jobs_archive_2017-01-01', '_source': {'produsername': 'gangarbt', 'jobstatus': 'finished', 'queue_time': 137, 'creationhost': 'hammercloud-ai-07.cern.ch', 'starttime': '2017-01-01T15:51:19', 'sourcesite': None, 'jobname': '73eb7f3d-fa62-4655-b354-8d95dd4ed850_90963', 'relocationflag': 1, 'failedattempt': None, 'maxcpucount': 0, 'pandaid': 3159633246, 'timeGetJob': 0, 'timeStageIn': 0, 'endtime': '2017-01-01T15:55:52', 'destinationsite': None, 'commandtopilot': None, 'transexitcode': '0', 'statechangetime': '2017-01-01T16:00:05', 'maxpss': 19934, 'timeSetup': 0, 'noutputdatafiles': 1, 'nucleus': None, 'exeerrorcode': 0, 'jobdispatchererrorcode': 0, 'jobsetid': None, 'taskbuffererrordiag': None, 'taskid': 1, 'taskbuffererrorcode': 0, 'assignedpriority': 0, 'dbData': None, 'cpuconsumptionunit': 's+Intel Core Processor (Haswell) 4096 KB', 'timeExe': 183, 'brokerageerrordiag': None, 'lockedby': None, 'batchid': '719459', 'cmtconfig': 'x86_64-slc6-gcc49-opt', 'com

In [ ]:
f=open("output.json", "w")
f.write(json.dumps(res['hits']['hits'], sort_keys=True, indent=2))
f.close()